In [2]:
import sys
import os
sys.path.append("..")
from tqdm import tqdm
import numpy as np
from utils import create_metadata_df
from dataset_xy import CARLADatasetXY
from data_preprocessing import preprocessing

from PIL import Image
import cv2


/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
path_data = os.path.join("..", "..", "..", "end2endappras","data", "data")

config_xy = {"used_inputs": ["lidar", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])
df_meta_data = df_meta_data.sort_values(by=["dir"])

# Create Dataset & DataLoader
dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)

Varying number files among input types: ../../../end2endappras/data/data/cycl_dataset_23_11/Routes_non-straight-junction-Scen4_Town07_junction_Seed2000/non-straight-junction-Scen4_Town07_junction_route105_11_24_01_59_00/lidar_bev
Varying number files among input types: ../../../end2endappras/data/data/rl_dataset_23_11/Routes_clipped_Town05_rl_Seed0/clipped_Town05_rl_route30_11_23_17_42_15/lidar_trans


In [7]:
df_meta = dataset.df_meta_data
for idx in tqdm(range(len(df_meta))):
    dir_new = os.path.join(dataset.df_meta_data["dir"][idx], "lidar_bev")
    if not os.path.exists(dir_new):
        os.makedirs(dir_new)
    path_load = os.path.join(df_meta.iloc[idx][0], "lidar", df_meta.iloc[idx]["lidar"])
    lidar_np = dataset.load_data_from_path(path_load)
    lidar_transformed_np = preprocessing["lidar"](lidar_np)
    lidar_img = Image.fromarray(lidar_transformed_np)
    path_save = os.path.join(dir_new, f"{df_meta.iloc[idx]['lidar'].split('.')[0]}{'.png'}")
    cv2.imwrite(path_save, lidar_transformed_np)

100%|██████████| 258816/258816 [20:53<00:00, 206.55it/s]
